# Extracción de características de los modelos obtenidos de BOLTZ-2  

Autor: Luis García Aguirre

### Prefacio:

Esta parte es la continuación de la generación de los modelos estructurales de los complejos del **dataset 22** con Blotz2. Paralelamente se modelaron los complejos con mutaciones y sin mutaciones. Estos pasos se pueden encontrar, respectivamente, en las carpetas:   

- [`22dataset_mutations_YAML` ](../22dataset_mutations_YAML/YAML_and_SH_maker.ipynb)

- [`22dataset_SIN_muts_YAML`](../22dataset_SIN_muts_YAML/YAML_and_SH_maker.ipynb)

Donde se generaron los `.yaml` y `job.sh` para correr Boltz-2. Una vez generados los modelos, nos interesa obtener los estadísticos pertinentes calculados durante el proceso; para tenerlos en cuenta a la hora de analizar datos o entrenar algoritmos de regresión o clasificación.  

Por una parte obtendremos las de los modelos sin mutaciones y por otra las de los modelos con mutaciones puntuales. La intención es añadir esta información a la tabla de datos que se realizó al inicio de estas prácticas obtenidos de la base de datos **RSBC PDB**; en virtud de aumentar la información disponible que permita una mejora en la creación de los algoritmos de regresión y clasificación.

* Al final de este parseo se visualizan ambos `.csv` y añaden columnas al final de este documento *markdown*.

### Paréntesis:

Las salidas de los modelos se guardaron en subcarpetas dentro las carpetas mencionadas. El siguiente script parte de la base de que estas carpetas se encuentren dentro del mismo directorio que esta carpeta. Esto es relevante, porque podría romper la reproducibilidad de las rutas. Se accederá a las salidas de otra carpeta, pero agrupación de estas características se guardará en esta carpeta como archivo `.csv`.

La extracción de características se produce separadamente, es decir, aquí, por cuestión de legibilidad y organización dentro de las otras carpetas. Además, aporta sensación de dirección por el hecho de que cada "gran paso" realizado en estas prácticas externas se produzca en una nueva carpeta.

## 1. Dataset con mutaciones

Extración de características globales de calidad de `confidence_{complejo_modelo}.json`, dentro de `predictions/`.

In [26]:
494 * 25

12350

Son 494 mutaciones en total sumando las mutaciones de todos los complejos. Se mandó generar 25 modelos por complejo-mutación en Boltz2, por lo que tendríamos que tener un total de 12350 modelos y entradas en nuestro `.csv` (sin contar cabecera).

**(IMPORTANTE) Detección de errores:**  

Es posible que algunos modelos **no** se hayan modelado por algún error con la conexión al servidor, falta de VRAM, etc. Por lo que, además de generar la ruta al `.json` también crearemos una lista con las posibles excepciones para crear de nuevo un `job.sh` que mandar de nuevo al servidor donde corremos Blotz2.

In [27]:
import os

In [28]:
ruta_modelos = '../22dataset_mutations_YAML/Models'

In [47]:
def get_boltz_json_paths(models_path, output_log):
    json_path_list = []
    exceptions_list = []
    log = []

    # Crear lista de rutas
    for result in os.listdir(models_path):
        try:
            cplx_name = result[14:]
            confidence_json_path = os.path.join(models_path, result, 'predictions', cplx_name)
            prediction_files = os.listdir(confidence_json_path)
            confid_json_list = [
                os.path.join(confidence_json_path, f) 
                for f in prediction_files if f.endswith('.json')
            ]
            json_path_list.extend(confid_json_list) # extend (no append) para no crear una lista de listas
            log.extend(confid_json_list)

        except Exception as e:
            exceptions_list.append(cplx_name)
            log.append(f"{cplx_name} - {str(e)}")
    
    # Escribir log
    with open(output_log+'.log', 'w') as log_file:
        texto = '\n'.join(log)
        log_file.write(texto)

    return json_path_list, exceptions_list

In [30]:
json_path_list, exceptions_list = get_boltz_json_paths(ruta_modelos, 'rutas_mutaciones')

En mutaciones.log podemos ver qué rutas se han guardado y cuales han dado error.

In [31]:
# Veamos qué excepciones tenemos:
exceptions_list.sort()
exceptions_list

['1C1Y_B.R13A',
 '1LFD_A.N21A',
 '3M62_B.I45A',
 '3M63_A.L44A',
 '4G0N_B.Q13A',
 '4OFY_A.D40A',
 '4OFY_A.E90A',
 '4OFY_A.F42A',
 '4OFY_A.M38A',
 '4OFY_A.Q36A',
 '4OFY_A.Q87A',
 '4OFY_A.S89A',
 '4OFY_B.L43A',
 '4OFY_B.Q35A',
 '4OFY_B.Q87A',
 '4OFY_B.R97A',
 '4RS1_B.F108A',
 '4RS1_B.R12D',
 '5F4E_A.D53A']

### Excepciones:

- Hay algunos modelos que no se han modelado del todo y faltan las predicciones. Generamos un archivo con los complejos que queremos remodelar.  

- Creamos de nuevo un job.sh para generar los modelos de nuevo (ir a [jobSH_REmaker](jobSH_REmaker.ipynb)):

In [32]:
with open('excepciones_ModConMuts_lista.txt', 'w') as salida:
    for linea in exceptions_list:
        salida.write(linea+'\n')

Continuamos con la generación de las tablas:

In [33]:
import os
import json
import csv

In [34]:
from collections import OrderedDict

def flatten_json(data):
    """Aplana las estructuras anidadas de un JSON."""
    flat = OrderedDict()
    for key, value in data.items():
        if isinstance(value, dict):
            for subkey, subval in value.items():
                if isinstance(subval, dict):
                    for subsubkey, final_val in subval.items():
                        flat[f"{key}_{subkey}_{subsubkey}"] = final_val
                else:
                    flat[f"{key}_{subkey}"] = subval
        else:
            flat[key] = value
    return flat

In [35]:
# Lista para almacenar todas las filas
all_rows = []
output_csv = "resultados_complejos_mutados.csv"

# Iterar sobre todos los archivos JSON
for filepath in json_path_list:
        with open(filepath, "r") as f:
            try:
                data = json.load(f)
                filename = os.path.basename(filepath).replace('.json', '')
                flat_data = {}
                flat_data["filename"] = filename  # Añadir nombre del archivo
                flat_data.update(flatten_json(data))
                all_rows.append(flat_data)

            except json.JSONDecodeError as e:
                print(f"Error en {filename}: {e}")

# Comprobación 5 primeras líneas
for i in all_rows[:5]:
     print(i)

{'filename': 'confidence_4RS1_A.K181A_model_13', 'confidence_score': 0.9315466284751892, 'ptm': 0.8883727192878723, 'iptm': 0.9358704686164856, 'ligand_iptm': 0.0, 'protein_iptm': 0.9358704686164856, 'complex_plddt': 0.930465579032898, 'complex_iplddt': 0.9363009929656982, 'complex_pde': 0.37610602378845215, 'complex_ipde': 0.7649507522583008, 'chains_ptm_0': 0.8651548624038696, 'chains_ptm_1': 0.9801903367042542, 'pair_chains_iptm_0_0': 0.8651548624038696, 'pair_chains_iptm_0_1': 0.8552778363227844, 'pair_chains_iptm_1_0': 0.9358704686164856, 'pair_chains_iptm_1_1': 0.9801903367042542}
{'filename': 'confidence_4RS1_A.K181A_model_12', 'confidence_score': 0.9318523406982422, 'ptm': 0.8826904296875, 'iptm': 0.9321122765541077, 'ligand_iptm': 0.0, 'protein_iptm': 0.9321122765541077, 'complex_plddt': 0.9317873120307922, 'complex_iplddt': 0.9372336268424988, 'complex_pde': 0.39187636971473694, 'complex_ipde': 0.814328670501709, 'chains_ptm_0': 0.8582340478897095, 'chains_ptm_1': 0.978904247

In [36]:
# Escribir CSV (sólo si hay filas válidas)
if all_rows:
    # Obtener todas las columnas únicas de todos los archivos
    fieldnames = [k for k in all_rows[0].keys()]

    with open(output_csv, "w", newline="") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in all_rows:
            writer.writerow({key: row.get(key, "") for key in fieldnames})

    print(f"CSV generado en: {output_csv}")
else:
    print("No se encontraron archivos JSON válidos.")


CSV generado en: resultados_complejos_mutados.csv


## 2. Dataset WT (sin mutaciones)

In [37]:
22*50

1100

Son 22 complejos en total. Se mandó generar 50 modelos en Boltz2, por lo que tendríamos que tener un total de 1100 modelos y entradas en nuestro `.csv` (sin contar cabecera).

In [38]:
ruta_modelos_wt = '../22dataset_SIN_muts_YAML/Models'

In [39]:
json_path_list_wt, exceptions_list_wt = get_boltz_json_paths(ruta_modelos_wt, 'rutas_wild_type')

In [40]:

# Veamos qué excepciones tenemos:
exceptions_list_wt.sort()
exceptions_list_wt


['3M62', '3M63', '4OFY']

### Excepciones:

- Hay algunos modelos que no se han modelado del todo y faltan las predicciones. Generamos un archivo con los complejos que queremos remodelar.  

- Creamos de nuevo un job.sh para generar los modelos de nuevo (ir a [jobSH_REmaker](jobSH_REmaker.ipynb)):

In [41]:
with open('excepciones_Mods_WT_lista.txt', 'w') as salida:
    for linea in exceptions_list_wt:
        salida.write(linea+'\n')

Continuamos con la generación de las tablas. Este código es copiapegado de arriba, podría hacer una función para no repetir código, pero en este caso prefiero dejarlo como está para realizar la comprobación entre medias de las primeras líneas. Esto facilita la depuración de errores con los modelos no modelados.

In [42]:
# Lista para almacenar todas las filas
all_rows = []
output_csv = "resultados_complejos_WT.csv"

# Iterar sobre todos los archivos JSON
for filepath in json_path_list_wt:
        with open(filepath, "r") as f:
            try:
                data = json.load(f)
                filename = os.path.basename(filepath).replace('.json', '')
                flat_data = {}
                flat_data["filename"] = filename  # Añadir nombre del archivo
                flat_data.update(flatten_json(data))
                all_rows.append(flat_data)

            except json.JSONDecodeError as e:
                print(f"Error en {filename}: {e}")

# Comprobación 5 primeras líneas
for i in all_rows[:5]:
     print(i)

{'filename': 'confidence_1KNE_model_1', 'confidence_score': 0.9136812090873718, 'ptm': 0.9622877836227417, 'iptm': 0.9403318166732788, 'ligand_iptm': 0.0, 'protein_iptm': 0.9403318166732788, 'complex_plddt': 0.9070186018943787, 'complex_iplddt': 0.9044320583343506, 'complex_pde': 0.2751876711845398, 'complex_ipde': 0.3443712890148163, 'chains_ptm_0': 0.9676922559738159, 'chains_ptm_1': 0.9408213496208191, 'pair_chains_iptm_0_0': 0.9676922559738159, 'pair_chains_iptm_0_1': 0.9279459118843079, 'pair_chains_iptm_1_0': 0.9403318166732788, 'pair_chains_iptm_1_1': 0.9408213496208191}
{'filename': 'confidence_1KNE_model_17', 'confidence_score': 0.9105048179626465, 'ptm': 0.9579930305480957, 'iptm': 0.946368932723999, 'ligand_iptm': 0.0, 'protein_iptm': 0.946368932723999, 'complex_plddt': 0.9015387892723083, 'complex_iplddt': 0.8857645392417908, 'complex_pde': 0.2779815196990967, 'complex_ipde': 0.349445641040802, 'chains_ptm_0': 0.9608756303787231, 'chains_ptm_1': 0.9404544830322266, 'pair_ch

In [43]:
# Escribir CSV (sólo si hay filas válidas)
if all_rows:
    # Obtener todas las columnas únicas de todos los archivos
    fieldnames = [k for k in all_rows[0].keys()]

    with open(output_csv, "w", newline="") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in all_rows:
            writer.writerow({key: row.get(key, "") for key in fieldnames})

    print(f"CSV generado en: {output_csv}")
else:
    print("No se encontraron archivos JSON válidos.")


CSV generado en: resultados_complejos_WT.csv


# 3. Visualización de los datos (se recomienda usar [Data Wrangler](https://code.visualstudio.com/docs/datascience/data-wrangler))

In [44]:
import pandas as pd

## Complejos Mutados:

In [45]:
df= pd.read_csv(output_csv)
df["Complex"]=df["filename"].str.split("_").str[1]
df["Mutation"]=df["filename"].str.split("_").str[2]
df["Complex_mutation"]=df["Complex"]+"_"+df["Mutation"]
df

,filename,confidence_score,ptm,iptm,ligand_iptm,protein_iptm,complex_plddt,complex_iplddt,complex_pde,complex_ipde,chains_ptm_0,chains_ptm_1,pair_chains_iptm_0_0,pair_chains_iptm_0_1,pair_chains_iptm_1_0,pair_chains_iptm_1_1,Complex,Mutation,Complex_mutation
0,confidence_1KNE_model_1,0.913681,0.962288,0.940332,0.0,0.940332,0.907019,0.904432,0.275188,0.344371,0.967692,0.940821,0.967692,0.927946,0.940332,0.940821,1KNE,model,1KNE_model
1,confidence_1KNE_model_17,0.910505,0.957993,0.946369,0.0,0.946369,0.901539,0.885765,0.277982,0.349446,0.960876,0.940454,0.960876,0.916120,0.946369,0.940454,1KNE,model,1KNE_model
2,confidence_1KNE_model_30,0.908534,0.963009,0.949181,0.0,0.949181,0.898372,0.888830,0.276718,0.349586,0.966412,0.942732,0.966412,0.923323,0.949181,0.942732,1KNE,model,1KNE_model
3,confidence_1KNE_model_7,0.912231,0.959346,0.947621,0.0,0.947621,0.903383,0.891381,0.274970,0.341611,0.962325,0.942646,0.962325,0.917534,0.947621,0.942646,1KNE,model,1KNE_model
4,confidence_1KNE_model_42,0.906410,0.956175,0.937653,0.0,0.937653,0.898600,0.884493,0.283104,0.377799,0.961240,0.934350,0.961240,0.917166,0.937653,0.934350,1KNE,model,1KNE_model
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,confidence_3EQY_model_11,0.968026,0.980068,0.975415,0.0,0.975415,0.966178,0.966236,0.263253,0.307328,0.980840,0.987263,0.980840,0.959629,0.975415,0.987263,3EQY,model,3EQY_model
946,confidence_3EQY_model_9,0.968070,0.976755,0.970996,0.0,0.970996,0.967338,0.967936,0.269857,0.337357,0.977676,0.984532,0.977676,0.954620,0.970996,0.984532,3EQY,model,3EQY_model
947,confidence_3EQY_model_23,0.967332,0.976272,0.969868,0.0,0.969868,0.966698,0.966946,0.268139,0.330556,0.977407,0.984338,0.977407,0.954620,0.969868,0.984338,3EQY,model,3EQY_model
948,confidence_3EQY_model_13,0.967879,0.976109,0.972520,0.0,0.972520,0.966719,0.968028,0.269257,0.319541,0.976838,0.985868,0.976838,0.955656,0.972520,0.985868,3EQY,model,3EQY_model


# Complejos Salvajes (wt):

In [46]:
df= pd.read_csv(output_csv)
df["Complex"]=df["filename"].str.split("_").str[1]
df["Mutation"]=df["filename"].str.split("_").str[2]
df["Complex_mutation"]=df["Complex"]+"_"+df["Mutation"]
df

,filename,confidence_score,ptm,iptm,ligand_iptm,protein_iptm,complex_plddt,complex_iplddt,complex_pde,complex_ipde,chains_ptm_0,chains_ptm_1,pair_chains_iptm_0_0,pair_chains_iptm_0_1,pair_chains_iptm_1_0,pair_chains_iptm_1_1,Complex,Mutation,Complex_mutation
0,confidence_1KNE_model_1,0.913681,0.962288,0.940332,0.0,0.940332,0.907019,0.904432,0.275188,0.344371,0.967692,0.940821,0.967692,0.927946,0.940332,0.940821,1KNE,model,1KNE_model
1,confidence_1KNE_model_17,0.910505,0.957993,0.946369,0.0,0.946369,0.901539,0.885765,0.277982,0.349446,0.960876,0.940454,0.960876,0.916120,0.946369,0.940454,1KNE,model,1KNE_model
2,confidence_1KNE_model_30,0.908534,0.963009,0.949181,0.0,0.949181,0.898372,0.888830,0.276718,0.349586,0.966412,0.942732,0.966412,0.923323,0.949181,0.942732,1KNE,model,1KNE_model
3,confidence_1KNE_model_7,0.912231,0.959346,0.947621,0.0,0.947621,0.903383,0.891381,0.274970,0.341611,0.962325,0.942646,0.962325,0.917534,0.947621,0.942646,1KNE,model,1KNE_model
4,confidence_1KNE_model_42,0.906410,0.956175,0.937653,0.0,0.937653,0.898600,0.884493,0.283104,0.377799,0.961240,0.934350,0.961240,0.917166,0.937653,0.934350,1KNE,model,1KNE_model
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,confidence_3EQY_model_11,0.968026,0.980068,0.975415,0.0,0.975415,0.966178,0.966236,0.263253,0.307328,0.980840,0.987263,0.980840,0.959629,0.975415,0.987263,3EQY,model,3EQY_model
946,confidence_3EQY_model_9,0.968070,0.976755,0.970996,0.0,0.970996,0.967338,0.967936,0.269857,0.337357,0.977676,0.984532,0.977676,0.954620,0.970996,0.984532,3EQY,model,3EQY_model
947,confidence_3EQY_model_23,0.967332,0.976272,0.969868,0.0,0.969868,0.966698,0.966946,0.268139,0.330556,0.977407,0.984338,0.977407,0.954620,0.969868,0.984338,3EQY,model,3EQY_model
948,confidence_3EQY_model_13,0.967879,0.976109,0.972520,0.0,0.972520,0.966719,0.968028,0.269257,0.319541,0.976838,0.985868,0.976838,0.955656,0.972520,0.985868,3EQY,model,3EQY_model
